In [6]:
import os
from typing import Tuple, Dict, List, Any
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import datetime

In [12]:
load_dotenv()

class CodeTranslationSystem:
    def __init__(self):
        self.super_agent = self.create_super_agent()
        self.translation_agent = self.create_translation_agent()
        self.max_iterations = 3  # Maximum iterations to prevent infinite loops
        self.execution_log: List[Dict[str, Any]] = []
    
    def _log_step(self, step_name: str, input_data: dict, output_data: str):
        """Log each step's input and output"""
        self.execution_log.append({
            "step": step_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "input": input_data,
            "output": output_data
        })

    def create_super_agent(self) -> LLMChain:
        """Create requirement analysis agent"""
        analysis_template = """You are a senior code analysis expert. Perform these tasks:
        1. Identify source programming language (C/C++/FORTRAN/CUDA/OpenMP/JAX)
        2. Identify target language (C/C++/FORTRAN/CUDA/OpenMP/JAX)
        3. Extract code content needing conversion
        4. Generate code conversion task description

        User input: {user_input}

        Respond in this format:
        Source Language: [detected source language]
        Target Language: [detected target language]
        Code Content: [extracted code block]
        Task Description: "Convert the following [source] code to [target]:\n[code]"
        """
        prompt = PromptTemplate(
            template=analysis_template,
            input_variables=["user_input"]
        )
        return LLMChain(
            llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
            prompt=prompt,
            output_key="analysis"
        )

    def create_translation_agent(self) -> LLMChain:
        """Create code translation agent"""
        translation_template = """You are an HPC code conversion expert. Convert this {source_lang} code to {target_lang}:
        Requirements:
        1. Maintain identical algorithmic logic
        2. Follow target language's performance best practices
        3. Add necessary comments explaining modifications
        4. Ensure syntactic correctness

        {code_input}

        Return ONLY converted code without explanations.
        """
        prompt = PromptTemplate(
            template=translation_template,
            input_variables=["source_lang", "target_lang", "code_input"]
        )
        return LLMChain(
            llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
            prompt=prompt,
            output_key="translated_code"
        )

    def parse_analysis(self, analysis: str) -> Tuple[str, str, str]:
        """Parse SuperAgent's analysis results"""
        source_lang = target_lang = code_content = ""
        for line in analysis.split('\n'):
            if line.startswith('Source Language:'):
                source_lang = line.split(': ')[1].strip()
            elif line.startswith('Target Language:'):
                target_lang = line.split(': ')[1].strip()
            elif line.startswith('Code Content:'):
                code_content = line.split(': ')[1].strip()
        return source_lang, target_lang, code_content

    def validate_code(self, code: str, target_lang: str) -> str:
        """Code validation workflow"""
        validation_template = """Review this {target_lang} code:
        {code}

        Format your findings as:
        Issues Found: [Yes/No]
        Issue List: 
        - [Issue1 description with line number]
        - [Issue2 description with line number]
        Suggestions: 
        - [Suggestion1]
        """
        prompt = PromptTemplate(
            template=validation_template,
            input_variables=["target_lang", "code"]
        )
        chain = LLMChain(
            llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
            prompt=prompt
        )
        result = chain.run(target_lang=target_lang, code=code)
        
        self._log_step(
            step_name="code_validation",
            input_data={"target_lang": target_lang, "code": code},
            output_data=result
        )
        
        return result

    def iterative_improvement(self, initial_code: str, target_lang: str) -> str:
        """Iterative code improvement"""
        current_code = initial_code
        iteration_log = []
        
        for iter_num in range(self.max_iterations):
            validation_result = self.validate_code(current_code, target_lang)
            
            iteration_data = {
                "iteration": iter_num + 1,
                "code_before": current_code,
                "validation_result": validation_result
            }
            
            if "Issues Found: No" in validation_result:
                iteration_data["action"] = "validation_passed"
                iteration_log.append(iteration_data)
                break

            improvement_prompt = f"""Improve the code based on this report:
            {validation_result}

            Original Code:
            {current_code}

            Requirements:
            1. Strictly follow the suggestions
            2. Preserve functionality
            3. Return complete revised code
            """
            self._log_step(
                step_name="improvement_request",
                input_data={
                    "validation_result": validation_result,
                    "previous_code": current_code
                },
                output_data=improvement_prompt
            )
            
            new_code = self.translation_agent.run({
                "source_lang": target_lang,
                "target_lang": target_lang,
                "code_input": improvement_prompt
            })
            
            self._log_step(
                step_name="improvement_response",
                input_data={"prompt": improvement_prompt},
                output_data=new_code
            )
            
            iteration_data.update({
                "action": "code_improved",
                "code_after": new_code
            })
            iteration_log.append(iteration_data)
            current_code = new_code
            
        self._log_step(
            step_name="iteration_summary",
            input_data={
                "initial_code": initial_code,
                "max_iterations": self.max_iterations,
                "target_language": target_lang
            },
            output_data={
                "final_code": current_code,
                "iterations": iteration_log,
                "total_iterations": len(iteration_log)
            }
        )
        
        return current_code

    def process_request(self, user_input: str) -> Dict:
        """Process user request with full logging"""
        self._log_step(
            step_name="user_input",
            input_data={},
            output_data=user_input
        )
        
        analysis = self.super_agent.run(user_input=user_input)
        self._log_step(
            step_name="requirement_analysis",
            input_data={"user_input": user_input},
            output_data=analysis
        )
        
        source_lang, target_lang, code_content = self.parse_analysis(analysis)
        
        self._log_step(
            step_name="analysis_parsing",
            input_data={"raw_analysis": analysis},
            output_data={
                "source_lang": source_lang,
                "target_lang": target_lang,
                "code_content": code_content
            }
        )

        translated_code = self.translation_agent.run({
            "source_lang": source_lang,
            "target_lang": target_lang,
            "code_input": code_content
        })
        
        self._log_step(
            step_name="initial_translation",
            input_data={
                "source": source_lang,
                "target": target_lang,
                "code": code_content
            },
            output_data=translated_code
        )

        final_code = self.iterative_improvement(translated_code, target_lang)

        return {
            "source_language": source_lang,
            "target_language": target_lang,
            "original_code": code_content,
            "translated_code": final_code,
            "execution_log": self.execution_log
        }

In [13]:
if __name__ == "__main__":
    system = CodeTranslationSystem()
    
    # Example Input
    user_input = """
    Please help me convert the following FORTRAN code into CUDA code:
    PROGRAM VECTOR_ADD
    INTEGER, PARAMETER :: N = 1000000
    REAL :: A(N), B(N), C(N)
    DO I = 1, N
        C(I) = A(I) + B(I)
    END DO
    END PROGRAM
    """
    
    result = system.process_request(user_input)
    print("Conversion Results:")
    print(result["translated_code"])
    
    print("\n=== Full Execution Log ===")
    for log in result["execution_log"]:
        print(f"\n[{log['step']}]")
        print("Input:", log.get("input"))
        print("Output:", log.get("output"))

Conversion Results:
```cuda
#include <stdio.h>

__global__ void vecAdd(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

void checkCudaError(cudaError_t error) {
    if (error != cudaSuccess) {
        fprintf(stderr, "CUDA error: %s\n", cudaGetErrorString(error));
        exit(1);
    }
}

int main() {
    int n = 100000;
    float *h_a, *h_b, *h_c;
    float *d_a, *d_b, *d_c;

    h_a = (float*)malloc(n * sizeof(float));
    h_b = (float*)malloc(n * sizeof(float));
    h_c = (float*)malloc(n * sizeof(float));

    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i * 2;
    }

    cudaMalloc(&d_a, n * sizeof(float));
    checkCudaError(cudaGetLastError());

    cudaMalloc(&d_b, n * sizeof(float));
    checkCudaError(cudaGetLastError());

    cudaMalloc(&d_c, n * sizeof(float));
    checkCudaError(cudaGetLastError());

    cudaMemcpy(d_a, h_a, n * sizeof(float), cudaMemcpyHos